**1. Importing Essential Libraries**

In [2]:
# initiating gpu using tensorflow.
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from google.colab import files

In [3]:
files.upload()

MessageError: ignored

**Reading all the three file**

In [7]:
gpu=pd.read_csv("gpu.csv")
application_checkpoints=pd.read_csv("application-checkpoints.csv")
task=pd.read_csv("task-x-y.csv")

**Descriptive stats for all the tree files**

In [30]:
application_checkpoints.describe()

,timestamp,hostname,eventName,eventType,jobId,taskId
count,660400,660400,660400,660400,660400,660400
unique,363555,1024,5,2,3,65793
top,2018-11-08T07:41:31.776Z,0d56a730076643d585f77e00d2d8521a00000P,TotalRender,START,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,1506221a-c292-49a8-88a5-9dc1ba99b9e9
freq,18,1240,132080,330200,657810,20


In [29]:
gpu.describe()

,gpuSerial,powerDrawWatt,gpuTempC,gpuUtilPerc,gpuMemUtilPerc
count,1.543681e+06,1.543681e+06,1.543681e+06,1.543681e+06,1.543681e+06
mean,3.239836e+11,8.919838e+01,4.007560e+01,6.305820e+01,3.341359e+01
std,1.228841e+09,3.975742e+01,3.800243e+00,4.144816e+01,2.300107e+01
min,3.201181e+11,2.255000e+01,2.600000e+01,0.000000e+00,0.000000e+00
25%,3.236170e+11,4.499000e+01,3.800000e+01,0.000000e+00,0.000000e+00
50%,3.236170e+11,9.659000e+01,4.000000e+01,8.900000e+01,4.300000e+01
75%,3.250170e+11,1.213400e+02,4.200000e+01,9.200000e+01,5.100000e+01
max,3.252171e+11,1.970100e+02,5.500000e+01,1.000000e+02,8.300000e+01


In [31]:
task.describe()

,x,y,level
count,65793.000000,65793.000000,65793.000000
mean,127.031143,127.031143,11.984314
std,74.135963,74.135963,0.250965
min,0.000000,0.000000,4.000000
25%,63.000000,63.000000,12.000000
50%,127.000000,127.000000,12.000000
75%,191.000000,191.000000,12.000000
max,255.000000,255.000000,12.000000


**seperating the start and stop time to determine which event type dominates the task run times**

In [12]:
def fun(accountdf):
    optimiseddf=pd.DataFrame()
    list_of_tasks=accountdf.taskId.unique()
    num=0
    for task in list_of_tasks:
        nec=accountdf[accountdf['taskId']==task]
        lis_of_events=['Tiling','Render','TotalRender','Uploading','Saving Config']
        print("task -"+str(num)+" name:"+task)
        for event in lis_of_events:
                k=nec[nec['eventName']==event]
                if(k['timestamp'][k.index[0]]=='START'):
                    newcol = {'timestamp1':k['timestamp'][k.index[0]],'timestamp2':k['timestamp'][k.index[1]],
                             'hostname':k['hostname'][k.index[0]],
                             'eventName':k['eventName'][k.index[0]],
                            'jobId':k['jobId'][k.index[0]],
                             'taskId':k['taskId'][k.index[0]]}

                else:
                    newcol = {'Start time':k['timestamp'][k.index[1]],'Stop time':k['timestamp'][k.index[0]],
                             'hostname':k['hostname'][k.index[0]],
                             'eventName':k['eventName'][k.index[0]],
                            'jobId':k['jobId'][k.index[0]],
                             'taskId':k['taskId'][k.index[0]],
                             }
                optimiseddf = optimiseddf.append(newcol, ignore_index=True)
        num=num+1           
    return optimiseddf

In [13]:
sorted_data=fun(application_checkpoints)

Streaming output truncated to the last 5000 lines.
task -60793 name:e5e8a65b-6695-40fc-9012-b6d04b4de5ee
task -60794 name:c44cd07a-0a62-48bd-b0e9-98637385122a
task -60795 name:61616710-c3c4-4ee7-bfb1-fcceb85c98cb
task -60796 name:42b558ad-54c5-401c-8daf-018a65b31b3b
task -60797 name:5e361dcc-baff-4cc2-b3f4-9eea825e4d66
task -60798 name:e60a099a-e20b-408f-81ec-e7c2d90fd6f1
task -60799 name:a48ac666-ea25-49a6-b622-178b05a30477
task -60800 name:2a45ef5f-b5de-4862-b228-e620c73bbcfe
task -60801 name:111f9a41-ef0f-43ff-a283-2e952e5011a2
task -60802 name:4b694f6d-d79e-404b-b2ca-346b3a0afe39
task -60803 name:a9c6eff7-ffcd-4720-93f6-3d28eedb626a
task -60804 name:719620e4-21f0-4f38-983b-c2c917d018c4
task -60805 name:10ad47ce-08d8-4c01-9f6e-ac8fe2ddde03
task -60806 name:5b3123d1-baee-4509-a308-7062778bf47d
task -60807 name:7df9fc6c-77ed-48d5-a207-28917a7ea55f
task -60808 name:c7dfb362-7447-48b7-8513-73abf105ddc2
task -60809 name:91ffeb0d-b7b9-45b1-965e-2a5140fc367b
task -60810 name:784a7c0d-a944-

**Saving the data into new csv file**

In [14]:
sorted_data.to_csv("sorted_df.csv")

**Reading the datafile into dataframe**

In [46]:
sorted_df2=pd.read_csv("sorted_df.csv")
sorted_df2.head()

,Unnamed: 0,Start time,Stop time,eventName,hostname,jobId,taskId
0,0,2018-11-08T07:41:55.200Z,2018-11-08T07:41:55.921Z,Tiling,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
1,1,2018-11-08T07:41:55.190Z,2018-11-08T07:41:31.689Z,Render,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
2,2,2018-11-08T07:41:56.111Z,2018-11-08T07:41:31.687Z,TotalRender,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
3,3,2018-11-08T07:41:56.111Z,2018-11-08T07:41:55.190Z,Uploading,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
4,4,2018-11-08T07:41:31.687Z,2018-11-08T07:41:31.689Z,Saving Config,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043


**Droping the unamed column from the dataset**

In [47]:
sorted_df2=sorted_df2.drop("Unnamed: 0",axis=1)

**Converting the start and stop time to proper data-time foramat**

In [48]:
sorted_df2[['Stop time','Start time']]=sorted_df2[["Start time","Stop time"]].apply(pd.to_datetime,format="%Y-%m-%d %H:%M:%S.%f")
sorted_df2.head()

,Start time,Stop time,eventName,hostname,jobId,taskId
0,2018-11-08 07:41:55.921000+00:00,2018-11-08 07:41:55.200000+00:00,Tiling,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
1,2018-11-08 07:41:31.689000+00:00,2018-11-08 07:41:55.190000+00:00,Render,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
2,2018-11-08 07:41:31.687000+00:00,2018-11-08 07:41:56.111000+00:00,TotalRender,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
3,2018-11-08 07:41:55.190000+00:00,2018-11-08 07:41:56.111000+00:00,Uploading,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043
4,2018-11-08 07:41:31.689000+00:00,2018-11-08 07:41:31.687000+00:00,Saving Config,0d56a730076643d585f77e00d2d8521a00000N,1024-lvl12-7e026be3-5fd0-48ee-b7d1-abd61f747705,b47f0263-ba1c-48a7-8d29-4bf021b72043


In [49]:
sorted_df2.dtypes

Start time    datetime64[ns, UTC]
Stop time     datetime64[ns, UTC]
eventName                  object
hostname                   object
jobId                      object
taskId                     object
dtype: object

**Calculating the time difference**

In [ ]:
sorted_df2["time difference"]=sorted_df2["Stop time"]-sorted_df2["start time"]